<a href="https://colab.research.google.com/github/fadodo/Flood_mapping/blob/main/Flood_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flood mapping from SAR imagery

## Loading of the necessaries package

In [1]:
## Loading of the necessaries package
import ee
import geemap

In [2]:
import geemap.chart as chart
import ee.mapclient
import plotly.express as px

## Authentification on Google Earth Engine

In [3]:
# Authenticate
ee.Authenticate()

# initialize
ee.Initialize(project='ee-fid')

## Definition of the area of interet for the studie

This can be done automatically by drawing the AOI on the map or defined manually the coordinates using geojson.io

In [ ]:
## First, plot the map
map=geemap.Map(zoom=7)
map

In [6]:
# AOI definition
if map.user_roi is not None:
    AOI=map.user_roi
else: # use the default AOI on
    AOI=ee.Geometry.Polygon(
        [
            [
            [-1.5731367943235819,5.3239250161140115],
            [0.9647050025514181,5.3239250161140115],
            [0.9647050025514181,8.942534511310892],
            [-1.5731367943235819,8.942534511310892],
            [-1.5731367943235819,5.3239250161140115]
            ]
        ]
    )


In [7]:
## Check the AOI coordinate
AOI.getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[1.002502, 6.109149],
   [1.002502, 6.337137],
   [1.812744, 6.337137],
   [1.812744, 6.109149],
   [1.002502, 6.109149]]]}

## Collect the copernicus SAR IW imagery

In [8]:
## Identifier la zone d'étude et les images SAR disponibles pour celle-ci
S1_collection= ee.ImageCollection('COPERNICUS/S1_GRD')\
.filter(ee.Filter.eq('instrumentMode','IW'))\
.filter(ee.Filter.eq('orbitProperties_pass','ASCENDING'))\
.filterBounds(AOI)\
.filterDate('2024-05-01', '2024-05-30')

In [ ]:
S1_collection

## Identifying pre- and post- event imageries
Identify pre-flood and flood images

In [10]:
## Images SAR for pre-event
SAR_non_flood_img = S1_collection.filterDate('2024-05-13', '2024-05-15').median()
map.addLayer(SAR_non_flood_img,{'bands':['VH', 'VV'],'min':-25,'max':0},'SAR_non_flood_img')

## Images SAR for post-event
SAR_flood_img = S1_collection.filterDate('2024-05-25', '2024-05-27').median()
map.addLayer(SAR_flood_img,{'bands':['VH', 'VV'],'min':-25,'max':0},'SAR_flood_img')

In [ ]:
SAR_flood_img.bandNames()

In [ ]:
#map.center_object(AOI)
map

## Reduction of SAR speckle effect by medium focal smoothing

In [13]:
# Speckle correction
smoothing_radius = 50

# Pre-event imagery
SAR_non_flood_img_smooth = SAR_non_flood_img.focal_mean(smoothing_radius, 'circle','meters').clip(AOI)

# Post event
SAR_flood_img_smooth = SAR_flood_img.focal_mean(smoothing_radius, 'circle', 'meters').clip(AOI)

In [ ]:
map.addLayer(SAR_non_flood_img_smooth, {'bands':['VH'],'min':-25,'max':0}, 'SAR_non_flood_img_smooth')
map.addLayer(SAR_flood_img_smooth, {'bands':['VH'],'min':-25,'max':0}, 'SAR_flood_img_smooth')
map

## Identify a threshold to classify land and water for each image

Here, draw a polygon, a small area of interest (aoi_hist), in the map viewer to generate an image histogram.

From the histogram, visually identify a threshold value to classify land and water zones.

In [15]:
AOI_hist=map.user_roi

In [ ]:
AOI_hist.getInfo()

In [24]:
def plot_sar_histogram(image: ee.Image, region: ee.Geometry, scale: int = 10):
    """
    Plots overlaid histograms for VV and VH bands of a SAR image using Plotly Express.

    Parameters:
        image (ee.Image): Earth Engine image with VV and VH bands.
        region (ee.Geometry): Area of interest geometry.
        scale (int): Resolution in meters (default is 10m for Sentinel-1).
    """
    import plotly.express as px
    import pandas as pd

    # Reduce region to get histograms for both bands
    histogram = image.reduceRegion(
        reducer=ee.Reducer.histogram(),
        geometry=region,
        scale=scale,
        bestEffort=True
    ).getInfo()

    # Extract histograms
    vv_hist = histogram['VV']
    vh_hist = histogram['VH']

    # Convert to pandas DataFrame
    df_vv = pd.DataFrame({
        'Backscatter': vv_hist['bucketMeans'],
        'Frequency': vv_hist['histogram'],
        'Band': 'VV'
    })

    df_vh = pd.DataFrame({
        'Backscatter': vh_hist['bucketMeans'],
        'Frequency': vh_hist['histogram'],
        'Band': 'VH'
    })

    df_combined = pd.concat([df_vv, df_vh])

    # Plot with Plotly
    fig = px.bar(df_combined,
                 x='Backscatter',
                 y='Frequency',
                 color='Band',
                 barmode='overlay',
                 title='SAR Histogram - VV and VH Bands')

    fig.update_layout(template='plotly_white')
    fig.show()

In [ ]:
## Plot histogram for SAR_non_flood_smooth
plot_sar_histogram(SAR_non_flood_img_smooth, AOI_hist)

In [ ]:
## Plot histogram for SAR_non_flood_smooth
plot_sar_histogram(SAR_flood_img_smooth, AOI_hist)

# Classify the water and land  area
Based on the tresholds previously defined visually from histograms, we can now classify for classes

In [67]:
non_flood_class = SAR_non_flood_img_smooth.select('VH').lt(-22) # all pixels with value less than -22 will be assigned as 0, otherwise 1
map.addLayer(non_flood_class,{'palette':['gray','blue']}, 'Pre flood class')

flood_class = SAR_flood_img_smooth.select('VH').lt(-18.5) # all pixels with value less than -18.5 will be assigned as 0, otherwise 1
map.addLayer(flood_class,{'palette':['gray','blue']}, 'Flood class')


In [ ]:
map

## Calculate the extend of the flood

In [69]:
# Subtraction of non-flooded image from flooded classified image

flooded_area = flood_class.subtract(non_flood_class).selfMask()

In [ ]:
map.addLayer(flooded_area,{'palette':['blue']},'flooded_area')
map

## Calculate pixel connectivity to eliminate isolated pixels, e.g. neighbors with less than 8 pixels

In [71]:
connections = flooded_area.connectedPixelCount()
flooded_area_conn = flooded_area.updateMask(connections.gte(8))
map.addLayer(flooded_area_conn,{'palette':['yellow']},'flooded_area_conn')

In [ ]:
map

## Mask out areas with slopes greater than 5% using a digital elevation model

In [73]:
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')

terrain = ee.Algorithms.Terrain(DEM).clip(AOI)

slope = terrain.select('slope')

map.addLayer(terrain.select('hillshade'), {},'hillshade');

In [74]:
#Mask out areas with a slope greater than 5% using a digital elevation model.
flooded_area_conn_topo = flooded_area_conn.updateMask(slope.lt(5))
map.addLayer(flooded_area_conn_topo,{'palette':['red']},'flooded_area_conn_topo')

In [ ]:
map

## Export the flood map to your Google Drive

In [81]:
# Export the result to google earth engine Assets
task= ee.batch.Export.image.toAsset(
    image= flooded_area_conn_topo,
    description= 'flood_map',
    region= AOI.getInfo()['coordinates'],
    maxPixels= 1e9,
    scale= 10,
    assetId='projects/ee-fid/assets/flood_may24_Togo_',  #
)
task.start()

In [85]:
task.status()

{'state': 'RUNNING',
 'description': 'flood_map',
 'priority': 100,
 'creation_timestamp_ms': 1744961890369,
 'update_timestamp_ms': 1744961999117,
 'start_timestamp_ms': 1744961895597,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'batch_eecu_usage_seconds': 177.734,
 'id': '7RDZLXQBF55YKK4AOMRHR7ND',
 'name': 'projects/ee-fid/operations/7RDZLXQBF55YKK4AOMRHR7ND'}

# Calculate SAR flooded area

In [ ]:
area_flooded= geemap.zonal_stats(flooded_area_conn_topo, AOI, scale=1000, statistics_type='SUM', return_fc=True)
geemap.ee_to_df(area_flooded)